<a href="https://colab.research.google.com/github/NARAEIM/tp2-da/blob/jennlee/used.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

plt.style.use("default")

MessageError: ignored

In [ ]:
product = pd.read_csv('/content/drive/MyDrive/co_note/project/data/product_final.csv')

click_stream = pd.read_csv('/content/drive/MyDrive/co_note/project/data/buy_split.csv')

customer = pd.read_csv('/content/drive/MyDrive/co_note/project/data/customer_final.csv')

transaction = pd.read_csv('/content/drive/MyDrive/co_note/project/data/transaction_final.csv')

목적: 중고 물품에 대한 preference를 충족시키고 있는지<br>
> - product: 중고 product 확인할수 없음 <br>
1. click_stream: search 에서 중고에 대한 검색을 했는지<br>
2. transaction: 중고 검색 후 add_to_cart 로 간 경우, 그 상품이 어떤 상품인지 상관없이 중고에대한 만족을 했다고 이해해도 되지 않을까?



In [ ]:
click_stream.head()

,session_id,event_name,event_time,traffic_source,product_id,quantity,item_price,payment_status,search_keywords,promo_code,promo_amount,customer_id,booking_id,quarter
0,44122af0-0bf5-407b-b3ff-22c082c61426_0,HOMEPAGE,2016-10-09 21:17:34.380487+00:00,MOBILE,NaN,0,0,NaN,NaN,0.0,0,50.0,097c5406-68c5-4ecc-8399-8745cb3ff0ca,NaN
1,44122af0-0bf5-407b-b3ff-22c082c61426_0,ADD_TO_CART,2016-10-09 21:28:40.380487+00:00,MOBILE,24639.0,1,255356,NaN,NaN,0.0,0,50.0,097c5406-68c5-4ecc-8399-8745cb3ff0ca,NaN
2,44122af0-0bf5-407b-b3ff-22c082c61426_0,HOMEPAGE,2016-10-09 21:30:22.380487+00:00,MOBILE,NaN,0,0,NaN,NaN,0.0,0,50.0,097c5406-68c5-4ecc-8399-8745cb3ff0ca,NaN
3,44122af0-0bf5-407b-b3ff-22c082c61426_0,HOMEPAGE,2016-10-09 21:42:14.380487+00:00,MOBILE,NaN,0,0,NaN,NaN,0.0,0,50.0,097c5406-68c5-4ecc-8399-8745cb3ff0ca,NaN
4,44122af0-0bf5-407b-b3ff-22c082c61426_0,ITEM_DETAIL,2016-10-09 21:54:20.380487+00:00,MOBILE,NaN,0,0,NaN,NaN,0.0,0,50.0,097c5406-68c5-4ecc-8399-8745cb3ff0ca,NaN


In [ ]:
# search_keywords 가 없는 데이터는 drop
click_stream.dropna(subset=['search_keywords'],inplace=True)
click_stream.shape

(1134598, 14)

In [ ]:
session_data = click_stream[click_stream['session_id'].str.contains('316ee816-b672-4d8f-a91a-c87803c0aa3c')]
session_data

,session_id,event_name,event_time,traffic_source,product_id,quantity,item_price,payment_status,search_keywords,promo_code,promo_amount,customer_id,booking_id,quarter
137,316ee816-b672-4d8f-a91a-c87803c0aa3c_2,SEARCH,2016-12-08 05:52:08.807044+00:00,MOBILE,NaN,0,0,NaN,Second Hand,0.0,0,429.0,839b59bb-38c2-4a7a-95dc-c08cef238ef2,NaN
143,316ee816-b672-4d8f-a91a-c87803c0aa3c_5,SEARCH,2016-12-08 12:59:33.807044+00:00,MOBILE,NaN,0,0,NaN,Second Hand,0.0,0,429.0,839b59bb-38c2-4a7a-95dc-c08cef238ef2,NaN
146,316ee816-b672-4d8f-a91a-c87803c0aa3c_8,SEARCH,2016-12-08 20:08:27.807044+00:00,MOBILE,NaN,0,0,NaN,Second Hand,0.0,0,429.0,839b59bb-38c2-4a7a-95dc-c08cef238ef2,NaN
151,316ee816-b672-4d8f-a91a-c87803c0aa3c_13,SEARCH,2016-12-09 08:09:14.807044+00:00,MOBILE,NaN,0,0,NaN,T-Shirt,0.0,0,429.0,839b59bb-38c2-4a7a-95dc-c08cef238ef2,NaN


In [ ]:
click_stream.search_keywords.isna().sum()

11328406

In [ ]:
12463004 - 11328406

1134598

In [ ]:
click_stream.search_keywords.unique()

array(['Shirts', "Woman's Bag", 'Dress', 'Second Hand', 'T-Shirt',
       'Shoes', 'Adidas Shoes', 'Work Bag', 'Nike Shoes', 'Trousers',
       'Sandal', 'Watch', 'Hat', 'House dress', 'Pajamas'], dtype=object)

In [ ]:
used_cs = click_stream[click_stream.search_keywords == 'Second Hand']
used_cs.head()

,session_id,event_name,event_time,traffic_source,product_id,quantity,item_price,payment_status,search_keywords,promo_code,promo_amount,customer_id,booking_id,quarter
111,a77907da-645a-4f7e-a685-d99232b733de_0,SEARCH,2016-10-11 04:11:45.752206+00:00,MOBILE,NaN,0,0,NaN,Second Hand,0.0,0,302.0,14e77d56-9bec-4d6e-8cc4-d56251581d02,NaN
137,316ee816-b672-4d8f-a91a-c87803c0aa3c_2,SEARCH,2016-12-08 05:52:08.807044+00:00,MOBILE,NaN,0,0,NaN,Second Hand,0.0,0,429.0,839b59bb-38c2-4a7a-95dc-c08cef238ef2,NaN
143,316ee816-b672-4d8f-a91a-c87803c0aa3c_5,SEARCH,2016-12-08 12:59:33.807044+00:00,MOBILE,NaN,0,0,NaN,Second Hand,0.0,0,429.0,839b59bb-38c2-4a7a-95dc-c08cef238ef2,NaN
146,316ee816-b672-4d8f-a91a-c87803c0aa3c_8,SEARCH,2016-12-08 20:08:27.807044+00:00,MOBILE,NaN,0,0,NaN,Second Hand,0.0,0,429.0,839b59bb-38c2-4a7a-95dc-c08cef238ef2,NaN
375,f794b5cb-1c0f-4246-b724-1ac1be07de1e_0,SEARCH,2016-10-02 10:07:15.824834+00:00,MOBILE,NaN,0,0,NaN,Second Hand,0.0,0,1065.0,3045e8a7-3252-48f9-8a75-8c7e6a76ca33,NaN


In [ ]:
len(used_cs) / len(click_stream)

0.19163615659467054

search 중 used 제품에 대한 검색이 차지하는 데이터 = 19% <br>
현준님 그래프 보면 중고에 대한 검색량 상승 -> 2020-08월부터 날짜로 확인해보기

In [ ]:
click_stream['event_time'] = pd.to_datetime(click_stream['event_time']).dt.to_period('M')
click_stream = click_stream[click_stream['event_time'] >= '2020-08']

used_cs['event_time'] = pd.to_datetime(used_cs['event_time']).dt.to_period('M')
used_cs = used_cs[used_cs['event_time'] >= '2020-08']

print(click_stream.shape, used_cs.shape)

<ipython-input-33-f6be86eca333>:1: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  click_stream['event_time'] = pd.to_datetime(click_stream['event_time']).dt.to_period('M')


(735162, 14) (140852, 14)


<ipython-input-33-f6be86eca333>:4: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  used_cs['event_time'] = pd.to_datetime(used_cs['event_time']).dt.to_period('M')


In [ ]:
len(used_cs) / len(click_stream)

0.1915931454563756

2020-08월부터 날짜로 확인했을 때도 중고에 대한 검색이 전체 검색의 약 19% 차지

In [ ]:
used_cs.booking_id.isna().sum()

0

## segment별 중고 검색어 비율

재은!

## (필요없음)미구매자 포함 전체 중고 search keywords확인해봄

In [ ]:
# 미구매자 포함된 click_stream 에서 검색어 비율
click_old = pd.read_csv('/content/drive/MyDrive/co_note/project/data/original_data/click_stream_new.csv')

In [ ]:
click_old.dropna(subset=['search_keywords'],inplace=True)
click_old.shape

used_cs_old = click_old[click_old.search_keywords == 'Second Hand']

len(used_cs_old) / len(click_old)

0.024230651872635872

In [ ]:
click_old['event_time'] = pd.to_datetime(click_old['event_time']).dt.to_period('M')
click_old = click_old[click_old['event_time'] >= '2020-08']

used_cs_old['event_time'] = pd.to_datetime(used_cs_old['event_time']).dt.to_period('M')
used_cs_old = used_cs_old[used_cs_old['event_time'] >= '2020-08']

<ipython-input-41-4f9b4de7c5f1>:1: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  click_old['event_time'] = pd.to_datetime(click_old['event_time']).dt.to_period('M')
<ipython-input-41-4f9b4de7c5f1>:4: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  used_cs_old['event_time'] = pd.to_datetime(used_cs_old['event_time']).dt.to_period('M')


In [ ]:
len(used_cs_old) / len(click_old)

0.02420918002573839